In [ ]:
# Server
import socket
import threading
import json
import base64
from cryptography.fernet import Fernet

# Generate and save encryption key
KEY = Fernet.generate_key()
with open("secret.key", "wb") as key_file:
    key_file.write(KEY)

# Load encryption key
def load_key():
    with open("secret.key", "rb") as key_file:
        return key_file.read()

KEY = load_key()
cipher = Fernet(KEY)

# User credentials database
USER_CREDENTIALS = {"user1": "password1", "user2": "password2"}

# Active users dictionary
connected_clients = {}

# TCP Server setup
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 65432))
server_socket.listen(5)
print("TCP Chat Server is running...")

# Function to broadcast messages to all clients except the sender
def broadcast(message, sender_socket):
    for client_socket in connected_clients.keys():
        if client_socket != sender_socket:
            try:
                client_socket.sendall(message)
            except:
                client_socket.close()
                del connected_clients[client_socket]

# Handle client connection and authentication
def handle_client(client_socket, client_address):
    try:
        # Receive authentication data from the client
        data = client_socket.recv(1024).decode()
        message = json.loads(data)
        
        if message["type"] == "auth":
            username = message["username"]
            password = message["password"]
            
            # Validate user credentials
            if username in USER_CREDENTIALS and USER_CREDENTIALS[username] == password:
                connected_clients[client_socket] = username
                response = json.dumps({"status": "success", "message": "Authentication successful."})
                client_socket.sendall(response.encode())
            else:
                response = json.dumps({"status": "error", "message": "Invalid credentials."})
                client_socket.sendall(response.encode())
                client_socket.close()
                return  # Exit function if authentication fails
        
        while True:
            data = client_socket.recv(1024).decode()
            if not data:
                break
            
            message = json.loads(data)

            # Handle weather data separately
            if message["type"] == "weather":
                print("\nReceived Weather Data:\n", message["content"])
            else:
                decrypted_msg = cipher.decrypt(base64.urlsafe_b64decode(message["content"].encode())).decode()
                print(f"{connected_clients[client_socket]}: {decrypted_msg}")
                broadcast(data.encode(), client_socket)
    except Exception as e:
        print("Error handling client:", e)
    finally:
        print(f"Client {client_address} disconnected")
        client_socket.close()
        del connected_clients[client_socket]

# Accept incoming client connections
while True:
    client_socket, client_address = server_socket.accept()
    threading.Thread(target=handle_client, args=(client_socket, client_address)).start()


TCP Chat Server is running...
user1: Hello, how's your day?
user2: Hey! My day's been good, just studying. You?
user1: I'm working on a Python project.

Received Weather Data:
 Temperature Comparison:
Goldsmiths, University of London: 11.1°C
British Library: 11.4°C
Difference: 0.30°C
